<a href="https://colab.research.google.com/github/spatank/CIS-530/blob/master/Homework%206/spp_text_gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
root_directory = 'drive/Shared drives/cis530_hw6/data'
!ls 'drive/Shared drives/cis530_hw6/data'

all-abcs.txt  eng-fra.txt  jane_austen.txt  names  trump-speeches-master


# Text Generation

In [5]:
!ls 'drive/Shared drives/cis530_hw6/data/'

all-abcs.txt  eng-fra.txt  jane_austen.txt  names  trump-speeches-master


In [6]:
import string
import random
import re

all_characters = string.printable
n_characters = len(all_characters)

root_directory = 'drive/Shared drives/cis530_hw6/data/'
text_directory = 'jane_austen.txt'

file = open(root_directory + text_directory, encoding = 'UTF-8', errors = 'ignore').read()
file_len = len(file)
print('file_len =', file_len)

file_len = 4373592


In [7]:
chunk_len = 200

def random_chunk():
  start_index = random.randint(0, file_len - chunk_len)
  end_index = start_index + chunk_len + 1
  return file[start_index:end_index]

print(random_chunk())

rcumstances
under which she felt them. Nay, had she been without his arm, she would
soon have known that she needed it, for she wanted strength for a two
hours' saunter of this kind, coming, as it gene


In [0]:
import torch
import torch.nn as nn
from torch.autograd import Variable

class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size, n_layers=1):
    super(RNN, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.n_layers = n_layers
        
    self.encoder = nn.Embedding(input_size, hidden_size)
    self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
    self.decoder = nn.Linear(hidden_size, output_size)
    
  def forward(self, input, hidden):
    input = self.encoder(input.view(1, -1))
    output, hidden = self.gru(input.view(1, 1, -1), hidden)
    output = self.decoder(output.view(1, -1))
    return output, hidden
    
  def init_hidden(self):
    return Variable(torch.zeros(self.n_layers, 1, self.hidden_size))

In [9]:
# Turn string into list of longs
def char_tensor(string):
  tensor = torch.zeros(len(string)).long()
  for c in range(len(string)):
    tensor[c] = all_characters.index(string[c])
  return Variable(tensor)

print(char_tensor('abcDEF'))

tensor([10, 11, 12, 39, 40, 41])


In [0]:
def random_training_set():    
  chunk = random_chunk()
  inp = char_tensor(chunk[:-1])
  target = char_tensor(chunk[1:])
  return inp, target

In [0]:
def evaluate(prime_str='A', predict_len=100, temperature=0.8):
  hidden = decoder.init_hidden()
  prime_input = char_tensor(prime_str)
  predicted = prime_str
  
  # Use priming string to "build up" hidden state
  for p in range(len(prime_str) - 1):
    _, hidden = decoder(prime_input[p], hidden)
  inp = prime_input[-1]
  
  for p in range(predict_len):
    output, hidden = decoder(inp, hidden)
        
    # Sample from the network as a multinomial distribution
    output_dist = output.data.view(-1).div(temperature).exp()
    top_i = torch.multinomial(output_dist, 1)[0]
    
    # Add predicted character to string and use as next input
    predicted_char = all_characters[top_i]
    predicted += predicted_char
    inp = char_tensor(predicted_char)

  return predicted

In [0]:
import time, math

def time_since(since):
  s = time.time() - since
  m = math.floor(s / 60)
  s -= m * 60
  return '%dm %ds' % (m, s)

In [0]:
def train(inp, target):
  hidden = decoder.init_hidden()
  decoder.zero_grad()
  loss = 0

  for c in range(chunk_len):
      output, hidden = decoder(inp[c], hidden)
      loss += criterion(output, target[[c]])

  loss.backward()
  decoder_optimizer.step()

  return loss.data / chunk_len

In [14]:
n_epochs = 2000
print_every = 100
plot_every = 10
hidden_size = 100
n_layers = 1
lr = 0.005

decoder = RNN(n_characters, hidden_size, n_characters, n_layers)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

start = time.time()
all_losses = []
loss_avg = 0

for epoch in range(1, n_epochs + 1):
  loss = train(*random_training_set())       
  loss_avg += loss

  if epoch % print_every == 0:
      print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
      print(evaluate('Wh', 100), '\n')

  if epoch % plot_every == 0:
      all_losses.append(loss_avg / plot_every)
      loss_avg = 0

 18s (100 5%) 2.2677]
Whe of anh oand ing ant wen at seivecet edes neme hopplen in, been
Ne and dimeette it anst thed wat o  

 38s (200 10%) 2.2846]
Whe rece in the renesion an tho wlepronting to had veamppr."

"I githat han the youd a sucher dis" who 

 56s (300 15%) 1.9988]
Wh-!"
"II with our and the
saik a doevelry on sure have and be pexpooent te care the
agist then in I d 

 14s (400 20%) 1.8200]
Whithe invenied accelfulsoned theled in tilmittiot whure would have sis perfore
to count
for a colless 

 32s (500 25%) 1.9626]
Whilly wideass of in of monlauge othing. She mowanter in alfted the seate. So more of the mright af an 

 50s (600 30%) 1.8790]
Whe Sering be Chave with hard your piturition thourise to olf ill a
mainer. Could Mrs. Norner, had be  

 8s (700 35%) 1.7199]
Wherful Craabfan anys! My fell to him
happuring of being toow her som, was you so do, the pursent it a 

 26s (800 40%) 2.0343]
Wht, must surmed the to the of to was enou propriot not laterse to ready sact corle

In [0]:
import math

def perplexity(input_text):
  softmax = nn.Softmax()
  full_input_tensor = char_tensor(input_text)
  hidden = decoder.init_hidden()
  prob = 0
  for i in range(len(input_text)-1):
      output, hidden = decoder(full_input_tensor[i], hidden)
      output = softmax(output)
      next_char = input_text[i+1]
      if next_char in all_characters:
          next_char = all_characters.index(next_char)
      else:
          next_char = 84
      prob_next = math.log(output[0][next_char])
      prob = prob + prob_next
  prob = -1/(len(input_text)-1)*prob
  prob = math.exp(prob)
  return prob

In [16]:
text_directory = 'jane_austen.txt'

chunk_len = 200

def random_chunk():
  start_index = random.randint(0, file_len - chunk_len)
  end_index = start_index + chunk_len + 1
  return file[start_index:end_index]

random_text = random_chunk()
print(perplexity(random_text))

6.095214842603305


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # Remove the CWD from sys.path while we load stuff.
